<div class="alert alert-block alert-success">
    <h1>DataLab HvA - serie Machine Learning Python - Tech report 6</h1>
</div>

<div class="alert alert-block alert-warning">
    <h2>K-fold crossvalidatie en grid search</h2>
</div>

### 0. Doel van deze les
Je leert hoe je voor diverse modellen waarin de waarde van zogenaamde *hyperparatmeters* gekozen (*getuned*) moet worden dit kan middels een methode van *grid search*. Om je hierbij middels tussenresultaten op je trainingsdata zuiver te laten leiden in een beslissing voor de beste keuze maak je gebruik van een methode die *k-fold cross validatie* heet. We leren je hier hoe je deze methoden kunt gebruiken in het package `sklearn`. 

Naast de recht-toe-recht aan grid search bespreken we ook (in sommige gevallen) efficiëntere algoritmen zoals:
- Random Search en
- Baysian Search

Deze les gaat *niet* over specifieke algoritmen, de modellen in dit Tech Report zijn zuiver illustratief. 

### 1. Introductie hyperparameters
Vaak is het nodig bij het optimaliseren van een machine learning model enkele *hyperparameters* vast te stellen. *Hyperparameters* moeten niet verward worden met de *modelcoëfficiënten* die in het optimalisatieproces van training worden bepaald. De *hyperparameters* van een model worden voorafgaande aan het trainen van een (definitief) model vastgesteld. Denk aan:  
- het *maximale diepte*, de *cost complexity* en de *methode om het informatiecriterium vast te stellen* voor een beslisboom (zie TR 7 en 8)
- het *aantal bomen* en *het aantal variabelen per boom* in een random forest (zie TR 9 en 10)  
- de *kostenparameters* en de *inproductkeuze + parameters voor de kernel* voor een Supprot Vector Machine (zie TR 11 en 12)  
- het aantal *layers*, *aantal neuronen per layer*, de *activiatiefunctie(s)* in een neuraal netwerk

Modelcoëficiënten daarentegen zijn bijvoorbeeld de waarden voor $\alpha$ en $\beta_i$ voor een regressiemodel: 
$$ y = \alpha + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_k x_k +\epsilon$$

Ga na dat een lineair regressiemodel verder geen hyperparameters heeft. 

Andere voorbeelden van modelcoëfficiënten zijn:
- de *gekozen variabelen met grenswaarden voor de splitsingen* in de vertakkingen van een beslisboom 
- de waarde van de de *gewichten* in een neuraal netwerk  

De waarden van deze modelcoëfficënten worden gevonden in het optimalisatiealgoritme dat werkzaam is bij het trainen van een specifiek modeltype, maar *voorafgaande daaraan* zijn de *hyperparameters* al gekozen. Het maak echter wel uit *hoe* je deze kiest: **de modelprestatie is ervan afhankelijk**. Hoe stellen we deze dan vast?

Wat **niet** mag in een CRISP-DM methodiek is dat je diverse waarden voor hyperparameters uitprobeert en van de getrainde modellen vervolgens telkens de prestatie op de *test data* observeert en dan dát model kiest dat hierop het beste presteert. Ga na dat je dan de testset als een *tweede trainingset* hebt laten functioneren waardoor je *geen* betrouwbare uitspraak kan doen m.b.t. de te verwachten modelprestatie op *werkelijk* nieuwe data!

### 2. K-fold cross validatie
Een oplossing voor het omzeilen van het voortijdige gebruik van de test dataset is om *binnen de traindata* een deel af te splitsen die *tijdelijk* dienst doet als testset. Deze *tijdelijke testset afkomstig uit de train data* noemen we een **validatie dataset**. In het vervolg gaan we er vanuit dat er een prestatiemaat (performance metric) gekozen is die modelscores onderling vergelijkt, meer daarover in *TR3 - Prestatiematen en modelscore*. 

#### Eerste poging
We zouden bijvoorbeeld als volgt kunnen werken:  
- splist 20% van de traindata af als validatie dataset   
- met de 80% resterende train dataset trainen we diverse modellen, we passen steeds hyperparameter(s) aan  
- bepaal de modelscore van ieder getraind model op de validatieset en  
- kies uiteindelijk de hyperparameter(combinatie) die de beste modelscore gaf  
- ! train met *deze* hyperparameter(combinatie) het definitieve model op de *complete train dataset*  
- stel de modelscore van dit eindmodel vast op de *test dataset*  
- rapporteer de modelscore etc... (aanpassingen aan het model **niet** meer mogelijk)

Dat zou een prima werkwijze kunnen zijn, maar bedenk:  
- de resterende 80% van de traindata bevat mogelijk significant minder informatie dan de oorspronkelijke 100%  
- de validatieset (20% van de traindata) zou wel eens niet zo representatief kunnen zijn voor een set nieuwe data  

#### k-fold cross validatie
Hierdoor kan de *toevallige* keuze van de split in *resterende train dataset* vs *validatie dataset* wel eens een vertekend beeld gaan geven van de te verwachten prestaties op een nieuwe dataset. Om dit effect te elimineren passen we de werkwijze hierboven een beetje aan. Het stappenplan hieronder is een beschrijving van de methode van k-fold cross validatie, met k = 5. Meer specifiek: de beschreven *Stap 2* is de k-fold cross validatie, het doorlopen van verschillende opties voor de hyperparameters wordt meer uitgediept in de sectie *grid search*.

- Stap 1
    - splist de traindata in (k=) 5 gelijke sets (allen dus 20% van de train dataset) - we noemen deze de 5 *folds*   
    - nummer deze sets als fold-1, fold-2, fold-3, fold-4 en fold-5  
- Stap 2  
    - kies een (combinatie van) waarde(n) voor de hyperparameter(s)
    - Stap 3  
        - Kies fold-1 uit als validatieset 
        - met de overige sets (fold-2, fold-3, fold-4 en fold-5, dus 80% resterende train dataset) trainen we een model met de gekozen hyperparameter(combinatie)
        - test de performance van dit model op de validatieset (nu dus fold-1) en  
        - leg deze performance vast  
    - herhaal Stap 3 maar dan achtereenvolgens met één van de andere folds als validatieset (en de overigen voor resterende train dataset)  
    - middel de 5 gevonden modelscores, deze score hoort bij de gekozen hyperparametercombinatie in Stap 2  
- Herhaal Stap 2 voor een andere (combinatie van) waarde(n) voor de hyperparameter(s)
- Kies uiteindelijk die (combinatie van) waarde(n) voor de hyperparameter(s) die het beste (gemiddelde) modelscore gaf
- Stap 4  
    - ! train met *deze* hyperparametercombinatie het definitieve model op de *complete train dataset*  
    - stel de modelscore van dit eindmodel vast op de *test dataset*  
    - rapporteer de modelscore etc... (aanpassingen aan het model **niet** meer mogelijk)

Lees [hier](https://scikit-learn.org/stable/modules/cross_validation.html) meer over de methode in `sklearn`, je vindt er ook verhelderende grafische voorstellingen van het boven beschreven proces. 

#### Voorbeeld in sklearn
`sklearn` heeft standaardmethoden die bovenstaande methode toepassen, je hoeft de stappen dus niet zelf stap voor stap uit te voeren. De in het voorbeeld boven gekozen waarde *k = 5* folds is een keuze, vaak de *default*. Ga wel na dat hoe groter k is, hoe vaker een model getraind zal worden per (combinatie van) hyperparameter(s) en dat de resterende traindata die daarbij wordt gebruikt ook groter per keer is. Voor sommige (complexe) modeltypen kan dit nogal veel van het geheugen en de processor van de computer vergen, houd daar rekening mee! 

We kiezen in het voorbeeld als volgt:  
- Stap 0:   
    - we kiezen als modeltype de *beslisboom voor classificatie*  (zie ook TR 8)  met   
    - performace metric voor de modelscore: de *accuracy*   
    - op een dataset `iris`
- Stap 1, we kiezen  
    - k = 6  
- Stap 2, we kiezen  
    - costcomplexity parameter = 1
    - als methode om het informatiecriterium vast te stellen: `gini`  
    - de maximale diepte van de boom: 3
    
We gaan hieronder *niet* verder met het kiezen van andere hyperparametercombinaties, daarover verderop meer bij de sectie *grid search*. We doorlopen Stap 2 (de k-fold cross validatie) voor slechts één gekozen set hyperparameters. 

In [ ]:
# Voor de dataset en plots
from sklearn.datasets import load_iris

# Voor het bewerken en inspecteren van de data
import pandas as pd

# Voor het afsplitsen van een train- en test dataset
from sklearn.model_selection import train_test_split

# Voor het modelleren
from sklearn.tree import DecisionTreeClassifier 

In [ ]:
# Laden van een standaard dataset uit package scikit learn
iris = load_iris()
X = pd.DataFrame(iris.data, columns = iris.feature_names)
y = pd.Series(iris.target)

# Split in training en test data 
# random_state kiezen zorgt voor reproduceerbaarheid van dit resultaat - gebruik dat ! Een andere waarde is uiteraard ook prima
# we kiezen voor een verhouden train - test van 70% - 30%. Andere keuzen kunnen ook!
train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.30,random_state=999)

In [ ]:
# methode cross validatie uit sklearn en metrcics
from sklearn.model_selection import cross_val_score
from sklearn import metrics

#Definitie Model Beslisboom voor classificatie, we kiezen:
dtc = DecisionTreeClassifier(ccp_alpha=0.2,            #costcomplexity parameter = 0.2
                             criterion = 'gini',       #als methode om het informatiecriterium vast te stellen: gini
                             max_depth = 3,            #de maximale diepte van de boom: 3
                             random_state=999)         # Dit is geen hyperparameter van het model, maar t.b.v. reproductie

# Het aanroepen van de k-fold Cross Validatie (CV)
scores = cross_val_score(dtc, X, y, cv=6, scoring='accuracy') #we kiezen k = 6 en modelscore: de accuracy

In [ ]:
print("De scores op de (k =) 6 verschillende folds waren: ", scores)
print("Het gemiddelde van de scores op de (k =) 6 verschillende folds is: ", scores.mean())
print("De standaarddeviatie van de scores op de (k =) 6 verschillende folds is: ", scores.std())

Het valt op dat de variatie van de scores op de verschillende folds relatief groot is, waarmee tevens is geillustreerd dat het 6-voudig herhalen en het bepalen van het gemiddelde een representatiever beeld van de modelprestatie geven.

De waarde van de gemiddelde score over de 6 folds zou niet ver van de score van het eindmodel op de testset moeten liggen. Je kunt dit eventueel zelf nagaan, maar wij blijven hier nog even van de testset af. Want nu hebben we onze resultaten van slechts één combinatie van hyperparameters, we willen ook andere combinaties langsgaan. Dit doen we op een systematische manier.

### 3. Grid search
Het systematisch doorlopen van verschillende combinaties van waarden van hyperparameters kan geautomatiseerd worden uitgevoerd in een zogenaamde *grid search*. Het *grid* waar het om gaat is een representatie van de combinaties van hyperparameterwaarden die je stuk-voor-stuk gaat uitproberen. 

We borduren voort op het voorbeeld hierboven. We zouden willen verkennen  
- Stap 2, we willen proberen  
    - costcomplexity parameter in waardebereik: 0.1, 1, 10, 100
    - diverse methoden om het informatiecriterium vast te stellen: "gini", "entropy", "log_loss" 
    - diverse maximale diepten van de boom: 2, 3, 4, 5
    
**Opmerking**: heel vaak *heb je geen idee* welke waarden in de buurt liggen van een optimale waarde. Neem in dat geval grote stappen (zoals we nu bij de *costcomplexity parameter* hebben gedaan), met in het achterhoofd dat we straks bij een gevonden optimale waarde wat verder zouden kunnen *inzoomen* (je kunt het proces zo vaak herhalen als je maar wil).

Deze parameterwaarden, oftwel ons 'grid', leggen we vast:

In [ ]:
#Vastleggen van grid van parameterwaarden voor hyperparametertuning
parameters1={"criterion" : ["gini", "entropy", "log_loss"],
            "ccp_alpha" : [ 0.1, 1, 10, 100],
               "max_depth" : [1, 2, 3, 4, 5],
           }

Vervolgens importeren we de methode Grid Search Cross Validatie van `sklearn`, lees meer over deze methode [hier](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Het doorlopen van een parametergrid en het vaststellen van modelscores via k-fold cross validatie wordt hierin gecombineerd.

Ga na dat we 3 x 4 x 5 = 60 gridpunten hebben waarmee we een model gaan trainen en evalueren op 6 fold-combinaties, dus dat we 60 x 6 = 360 modelevaluaties gaan doorlopen.

In [ ]:
# Importeer vanuit scikit learn package de tooling:
from sklearn.model_selection import GridSearchCV

Het vastleggen / definieren van dit proces met de eerder genoemde keuzes gebeurt hieronder. Merk op dat het model zelf (`dtc`) heel algemeen wordt gedefinieerd en dat pas in de `GridSearchCV` verschillende parameterwaarden worden aangereikt (via `param_grid = parameters`)


In [ ]:
dtc = DecisionTreeClassifier(random_state=999)           # Dit is geen hyperparameter van het model, maar t.b.v. reproductie
tuning_model = GridSearchCV(estimator = dtc, 
                            param_grid = parameters1,    # we doorlopen het zojuist gedefinieerde grid 'parameters1'
                            scoring='accuracy',          # we kiezen modelscore: de accuracy
                            cv=6,                        # we kiezen k = 6 in de k-fold CV
                            verbose=1)                   # hiermee regel je de details van de output gedurende het trainen

Hieronder starten we de evaluatie, dit kan soms lang duren!

In [ ]:
tuning_model.fit(train_X,train_y)

We vragen ons af wat de beste combi is en de beste score:

In [ ]:
print("Beste model: ", tuning_model.best_estimator_)
print("Beste score:  accuracy = ", tuning_model.best_score_)
print("Beste parameter combi: ", tuning_model.best_params_)

#### Interpretatie eerste resultaten
- 1. Het valt op dat de optimale `ccp_alpha=0.1` op de *rand* van het gekozen bereik zit  
    - Nu zullen we ook verder waarden < 0.1 moeten verkennen (negatieve waarden kunnen hierbij in dit geval niet)
- 2. `max_depth = 3` (midden in het interval) en `criterion='entropy` zijn geoptimaliseerd terwijl we verder hadden moeten kijken voor `ccp_alpha`. We doen er goed aan hier voorlopig *alle opties open te houden* 

Zie hieronder

In [ ]:
#Vastleggen van grid van aangepaste parameterwaarden voor hyperparametertuning
parameters2={"criterion" : ["gini", "entropy", "log_loss"],
            "ccp_alpha" : [ 0, 0.1, 0.2, 0.3],
               "max_depth" : [1, 2, 3, 4, 5],
           }

In [ ]:
dtc = DecisionTreeClassifier(random_state=999)           # Dit is geen hyperparameter van het model, maar t.b.v. reproductie
tuning_model = GridSearchCV(estimator = dtc, 
                            param_grid = parameters2,    # we doorlopen het zojuist gedefinieerde grid 'parameters2'
                            scoring='accuracy',          # we kiezen modelscore: de accuracy
                            cv=6,                        # we kiezen k = 6 in de k-fold CV
                            verbose=1)                   # hiermee regel je de details van de output gedurende het trainen

In [ ]:
tuning_model.fit(train_X,train_y)

In [ ]:
print("Beste model: ", tuning_model.best_estimator_)
print("Beste score:  accuracy = ", tuning_model.best_score_)
print("Beste parameter combi: ", tuning_model.best_params_)

#### Interpretatie tweede resultaten
- We zien nu dat `DecisionTreeClassifier(ccp_alpha=0, max_depth=4, random_state=999)` het optimale model is  
- met een beste (gemiddelde) accuracy op de *validatiesets* van accuracy =  0.96 (hoger dan eerder, gan na!)  
- De waarden van `ccp_alpha=0` zit weer op de rand, maar deze waarde kan niet lager gekozen worden, dus hier eindigt de zoektocht  
- De waarde `max_depth=4` is anders vastgesteld dan boven, dus mooi dat die niet te vroeg werd vastgezet! De waarde zit *niet* op de rand van het gekozen interval, deze parameter kan alleen geheeltallig zijn  
- De waarde `'criterion': 'gini'` is anders vastgesteld dan boven, dus mooi dat die niet te vroeg werd vastgezet! - NB: omdat het de *default* is voor dit modeltype wordt deze niet expliciet vermeld bij `best_estimator_`

Op basis van deze gevonden combinatie definieren we het eindmodel en doorlopen Stap 4:

In [ ]:
#Definitie
dtc_opt = DecisionTreeClassifier(ccp_alpha=0, criterion = 'gini', max_depth=4, random_state=999) # criterion = 'gini' kan weggelaten

#fitten op hele train dataset
dtc_opt.fit(train_X,train_y)

In [ ]:
# Voorspellen op de test dataset - NB: wordt hier dus voor het eerst gebruikt!
predicted_y = dtc_opt.predict(test_X)

#accuracy op test dataset
print("De accuracy op de testset = ", metrics.accuracy_score(test_y, predicted_y))

Ga na dat de gevonden waarde op de test dataset niet veel afwijkt van de (gemiddelde) accuracy op de validatiesets van dit model in de grid search CV routine!

### 4 Werken met subset van de traindata
Een inituïtief overzichtelijke manier om de grid search - die soms heel tijdrovend kan zijn als het model veel parameters heeft, het grid heel groot is, het aantal folds groot gekozen wordt en/of de traindataset erg groot, is het om de eerste stappen van de gridsearch te zetten o.b.v. een deel van de trainingsdata. Hieronder zie je hoe je bijvoobeeld de sets `subset_train_X`en `subset_train_y` kan aanmaken die uit een random selectie van 10% van de oorspronkelijke set bestaat. Je kun vervolgens de grid search uitvoeren zoals je wilt maar dan met `subset_train_X`en `subset_train_y` in plaats van `train_X`en `train_y`. 

**Let op**: als de oorspronkelijke set al niet zo erg groot is, kan dit een risico met zich meebrengen (niet representatieve set)! Voor het bovenstaande voorbeeld is er geen enkele aanleiding deze stap te zetten, de set is zelf al klein en het model simpel!

In [ ]:
# Met het oog op de lange doorlooptijd gebruiken we een subsample van de train data voor het CV-grid search proces: 
# we kiezen om hiervoor met 30% van de data te werken - dit is een risico...
import numpy as np
n_subset = int(np.floor(len(train_X)*0.3)) #0.3 = 30% van de data 

from sklearn.utils import resample
subset_train_X, subset_train_y = resample(train_X,train_y, n_samples = n_subset, random_state=0)

print(subset_train_X.shape)
print(subset_train_y.shape)

### 5. Random Search  
Waar je nogal eens tegenaan kunt lopen in het proces van grid search is dat  
- sommige modellen per keer heel lang trainen (oorzaak kan de hoeveelheid data zijn en/of de complexiteit van het algoritme)  
- dat sommige modeltypen véél parameters kennen die je een grid search zou willen betrekken, het aantal te evalueren modellen loopt snel hoog op
- je aanvankelijk niet weet in welke range de optimale parameterwaarde van een specifieke parameter gezocht moet worden, hierdoor volg je vaak een iteratieve procedure: groot bereik en later (stapsgewijs) steeds meer inzoomen rondom gevonden optimale waarde  
- de methode om een deel van de traindata te gebruiken (vorige sectie) niet wenselijk is of te weinig oplost aan het probleem van heel lange evaluatietijd  

Dit kan een heel tijdrovende zaak worden!

Hier kan het uitkomst bieden als niet het *gehele* grid doorlopen wordt, maar een (random) gekozen selectie. Dit geeft in ieder geval vaak een eerste idee van de locatie binnen het grid waar het  optimum gevonden kan worden. 

Omdat er vaak ook parameters voor een modeltype bestaan die niet discreet, maar *continu* zijn, zou je in deze random procedure liever een verdeling dan een grid van een vast aantal punten willen meegeven. Dit kan door gebruik te maken van een verdelingsfunctie, hieronder ter illustratie een voorbeeld van de uniforme verdeling uit het package `scipy.stats`. Lees [hier](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) eventueel meer info over deze functie. 

<div class="alert alert-block alert-danger">
    <h2>Over onderstaande voorbeeld</h2>
    We willen vooral wat codevoorbeelden geven en suggesties hoe te werk te gaan. Voor het voorbeeldmodel en dataset is het niet nodig om deze techniek te gebruiken, een heel fijnmazig grid runt nog steeds heel snel. De resultaten zijn ook niet spectaculair, maar voor meer complexe modellen en grotere datasets (zie bijvoorbeeld Random Forest modellen en Support vector Machines in andere Tech Reports uit deze serie) loont dit zeker de moeite! Probeer daar deze technieken vooral uit!
</div>

In [ ]:
# Initieel definieren van het basismodel voor de grid search:
dtc_rand = DecisionTreeClassifier(random_state=999)     

# importeren uniform functie van package
from scipy.stats import uniform

#Vastleggen van grid van parameterwaarden voor hyperparametertuning, waarbij continue parameters als verdeling
parameters_rand={"criterion" : ["gini", "entropy", "log_loss"],
             "ccp_alpha" : uniform(loc=0, scale=2),
             "max_depth" : [1, 2, 3, 4, 5, 6, 7, 8],
             "min_samples_split" : uniform(loc=0, scale=0.001),
             "min_samples_leaf" : uniform(loc=0, scale=0.01),
           }

In scikit-learn is een functie `RandomizedSearchCV` die zeer verwant is aan de eerder gebruikte `GridSearchCV`, de parameters komen voor een belangrijk deel overeen, maar eentje is met name van belang: `n_iter = ... ` (defaulwaarde = 10). Hiermee kiezen we het aantal random punten uit het grid waarop we een model willen evalueren. Lees ook [deze info](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

In [ ]:
# Randomized grid search definieren met n_iter = 100 punten uit parameters_rand
from sklearn.model_selection import RandomizedSearchCV
tuning_model_rand  = RandomizedSearchCV(estimator = dtc_rand, param_distributions = parameters_rand, 
                                        random_state=9, 
                                        n_iter = 100, 
                                        scoring='accuracy',
                                        cv=6,
                                       verbose = 1)

In [ ]:
# Start het proces
tuning_model_rand.fit(train_X,train_y)

In [ ]:
results = pd.DataFrame(tuning_model_rand.cv_results_).sort_index(axis=1)[['rank_test_score','mean_test_score','param_ccp_alpha', 'param_criterion', 'param_max_depth',
       'param_min_samples_leaf', 'param_min_samples_split']].sort_values(by=['rank_test_score'])
results.head(10)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#create seaborn boxplots by group
sns.boxplot(data=results, x='param_criterion', y='mean_test_score')

In [ ]:
#results[results["param_criterion"]=="entropy"]
hm1 = sns.heatmap(results[results["param_criterion"]=="entropy"].pivot(index = 'param_max_depth', columns = 'param_ccp_alpha', values = 'mean_test_score'))

Een visualisatie van de ligging van de optimale waarden voor `'param_max_depth'` en `'param_ccp_alpha'` per gekozen split criterium: 

In [ ]:
#print(sns.heatmap(results[results["param_criterion"]=="entropy"].pivot(index = 'param_max_depth', columns = 'param_ccp_alpha', values = 'mean_test_score')))
#print(sns.heatmap(results[results["param_criterion"]=="log_loss"].pivot(index = 'param_max_depth', columns = 'param_ccp_alpha', values = 'mean_test_score')))
print(sns.heatmap(results[results["param_criterion"]=="gini"].pivot(index = 'param_max_depth', columns = 'param_ccp_alpha', values = 'mean_test_score')))

Een visualisatie van de ligging van de optimale waarden voor `'param_min_samples_leaf'` en `'param_min_samples_split'` per gekozen split criterium: 

In [ ]:
#print(sns.heatmap(results[results["param_criterion"]=="entropy"].pivot(index = 'param_min_samples_leaf', columns = 'param_min_samples_split', values = 'mean_test_score')))
print(sns.heatmap(results[results["param_criterion"]=="log_loss"].pivot(index = 'param_min_samples_leaf', columns = 'param_min_samples_split', values = 'mean_test_score')))

#print(sns.heatmap(results[results["param_criterion"]=="gini"].pivot(index = 'param_min_samples_leaf', columns = 'param_min_samples_split', values = 'mean_test_score')))

Uit bovenstaande maken we op dat de modellen met de `' criterion = gini'` het doorgaans minder goed doen, de andere twee opties ontlopen elkaar niet veel. Bij `' criterion = log_loss'` liggen de beste waarden voor `'param_min_samples_leaf'` en `'param_min_samples_split'` rond de 0.003 resp. zo klein mogelijken de `param_ccp_alpha` kan best zo klein mogelijk (0) worden gekozen en `'param_max_depth'` rond de 6. 

In [ ]:
# beste uit de bus vanuit random search: 
tuning_model_rand.cv_results_['params'][tuning_model_rand.best_index_]

In [ ]:
tuning_model_rand.best_score_

#### Vervolgstappen
Nu we een idee van de ligging van de parameters hebben waar de modelscore op de validatiesets het gemiddeld het beste lijkt te doen, zou een stap kunnen zijn in te zoomen in dat gebied met een gewone grid search. 

In [ ]:
#Definitie
dtc_opt_rand = tuning_model_rand.best_estimator_

#fitten op hele train dataset
dtc_opt_rand.fit(train_X,train_y)

Maar dan nu een grid search in een kleiner gebied. De parameter `"criterion"` kiezen we vast `"log_loss"` evenals de `"ccp_alpha" = 0`

In [ ]:
# Rondom gevonden gebied, kleine stappen
parameters3={"max_depth" : [3, 4, 5, 6] ,
             "min_samples_leaf" : [0.0001, 0.0005, 0.001, 0.0015, 0.002, 0.0025, 0.003, 0.0035, 0.004, 0.0045]
           }
dtc_3 = DecisionTreeClassifier(criterion = "log_loss", ccp_alpha = 0,  random_state=999)


In [ ]:
tuning_model_3 = GridSearchCV(estimator = dtc_3, 
                            param_grid = parameters3,    # we doorlopen het zojuist gedefinieerde grid 'parameters2'
                            scoring='accuracy',          # we kiezen modelscore: de accuracy
                            cv=6,                        # we kiezen k = 6 in de k-fold CV
                            verbose=1)                   # hiermee regel je de details van de output gedurende het trainen

In [ ]:
tuning_model_3.fit(train_X,train_y)

In [ ]:
print("Beste model: ", tuning_model_3.best_estimator_)
print("Beste score:  accuracy = ", tuning_model_3.best_score_)
print("Beste parameter combi: ", tuning_model_3.best_params_)

In [ ]:
#Definitie
dtc_opt_3 = tuning_model_3.best_estimator_

#fitten op hele train dataset
dtc_opt_3.fit(train_X,train_y)

#### Voorspelling op de test set
Het nu geoptimaliseerde model testen we op de testset. Denk er wel aan het model nog eenmaal op de complete train dataset te trainen!

In [ ]:
#Definitie
dtc_opt_3 = tuning_model_3.best_estimator_

#fitten op hele train dataset
dtc_opt_3.fit(train_X,train_y)

In [ ]:
# Voorspellen op de test dataset - NB: wordt hier dus voor het eerst gebruikt!
predicted_3_y = dtc_opt_3.predict(test_X)

#accuracy op test dataset
print("De accuracy op de testset = ", metrics.accuracy_score(test_y, predicted_3_y))



### 6. Baysian Search 
**< To do >**